In [85]:
import pandas as pd 
import numpy as np

# Create a dictionary to define the necessary month and year numbers
month_book = {
    'январь'   :  1,
    'февраль'  :  2,
    'март'     :  3,
    'апрель'   :  4,
    'май'      :  5,
    'июнь'     :  6,
    'июль'     :  7,
    'август'   :  8,
    'сентябрь' :  9,
    'октябрь'  : 10,
    'ноябрь'   : 11,
    'декабрь'  : 12
}
year_book = {
    '2010' : 2010,
    '2011' : 2011,
    '2012' : 2012,
    '2013' : 2013,
    '2014' : 2014,
    '2015' : 2015,
    '2016' : 2016,
    '2017' : 2017,
    '2018' : 2018,
    '2019' : 2019,
    '2020' : 2020,
    '2021' : 2021,
    '2022' : 2022,
    '2023' : 2023,
    '2024' : 2024,
    '2025' : 2025,
}

# Open the file
# Read the line with the name of the month
current_month_number_df = pd.read_excel(
    'D:/Work/python/Преобразования через pandas/по работе/Файлы для обработки/Производство/Н-п_53_10_1029.xlsx', 
    engine='openpyxl',
    sheet_name='Н-П_08',
    usecols='A:J', 
    header=0, 
    nrows=1
)
cols = current_month_number_df.columns[0]
cols = cols.split()

# Dictionary number of the year
for key in year_book:
    if key == cols[6]:
        current_year = year_book[key]
        previous_year = current_year - 1
        
# Dictionary number of the month        
for key in month_book:
    if key == cols[5]:
        current_month_index = month_of_the_previous_year_index = month_book[key]
        previous_month_index = current_month_index - 1

# Check if the year and month are correct        
print(f'Текущий месяц          {current_month_index} {current_year}')
print(f'Предыдущий месяц       {previous_month_index} {current_year}')
print(f'Месяц предыдущего года {month_of_the_previous_year_index} {previous_year}')        

# We carry out pre-processing of the data
pre_processed_df = pd.read_excel(
    'D:/Work/python/Преобразования через pandas/по работе/Файлы для обработки/Производство/Н-п_53_10_1029.xlsx', 
    engine='openpyxl',
    sheet_name = ['Н-П_08', 'Н-П_16', 'Н-П_17', 'Н-П_22', 'Н-П_23', 'Н-П_24', 'Н-П_31'],
    usecols='A:J', header=7
)
pre_processed_df = pd.concat(pre_processed_df)
pre_processed_df = pre_processed_df.rename(columns={'Unnamed: 0' : 'production',      
                                                    'Unnamed: 1' : 'code',
                                                    'Unnamed: 2' : 'current_month',
                                                    'Unnamed: 3' : 'previous_month',
                                                    'Unnamed: 4' : 'month_of_the_previous_year',
                                                    'Unnamed: 5' : 'period_since_the_beginning_of_the_reporting_year',
                                                    'Unnamed: 6' : 'corresponding_period_from_the_beginning_of_the_previous_year'})
pre_processed_df = pre_processed_df[['production', 
                                     'code', 
                                     'current_month', 
                                     'previous_month', 
                                     'month_of_the_previous_year',
                                     'period_since_the_beginning_of_the_reporting_year',
                                     'corresponding_period_from_the_beginning_of_the_previous_year']]
pre_processed_df['index'] = (pd.isna(pre_processed_df['current_month']) 
                           & pd.isna(pre_processed_df['previous_month']) 
                           & pd.isna(pre_processed_df['month_of_the_previous_year']) 
                           & pd.isna(pre_processed_df['period_since_the_beginning_of_the_reporting_year']) 
                           & pd.isna(pre_processed_df['corresponding_period_from_the_beginning_of_the_previous_year']))
pre_processed_df['country']  = 0
pre_processed_df['country']  = pre_processed_df.loc[pre_processed_df['index'] == False]
pre_processed_df['new_code'] = np.where(pre_processed_df['code'].str.find('.') == 2, pre_processed_df.code, np.NaN)
pre_processed_df['new_code'].fillna(method='ffill', inplace=True)

# Process the data for the current month of the current year.
df = pre_processed_df.copy()
df = df[['country', 'current_month', 'new_code']]
df['Код ОКПД'] = ''
df['Параметр'] = 'Производство'
df['Месяц']    = current_month_index
df['Год']      = current_year
df['1, 2, 3']  = 1                       #  current month index
df = df.rename(columns={'country' : 'Субъект',
                        'current_month' : 'Значение',
                        'new_code' : 'Код ОКПД-2'})
df = pd.DataFrame(df, columns=['Субъект',
                               'Код ОКПД',
                               'Параметр',
                               'Значение',
                               'Месяц',
                               'Год',
                               '1, 2, 3',
                               'Код ОКПД-2'])
df = df.dropna(subset=['Субъект'])
writer = pd.ExcelWriter('D:/Work/python/Преобразования через pandas/\
по работе/Обработанные файлы/Необходимые коды/Текущий месяц.xlsx')
df.to_excel(writer)
writer.save()
print('Данные обработаны. Текущий месяц загружен в файл')

# Process the data for the previous month of the current year.
df = pre_processed_df.copy()
df = df[['country', 'previous_month', 'new_code']]
df['Код ОКПД'] = ''
df['Параметр'] = 'Производство'
df['Месяц']    = previous_month_index
df['Год']      = current_year
df['1, 2, 3']  = 2                       #  previous month index
df = df.rename(columns={'country' : 'Субъект',
                        'previous_month' : 'Значение',
                        'new_code' : 'Код ОКПД-2'})
df = pd.DataFrame(df, columns=['Субъект',
                               'Код ОКПД',
                               'Параметр',
                               'Значение',
                               'Месяц',
                               'Год',
                               '1, 2, 3',
                               'Код ОКПД-2'])
df = df.dropna(subset=['Субъект'])
writer = pd.ExcelWriter('D:/Work/python/Преобразования через pandas/\
по работе/Обработанные файлы/Необходимые коды/Предыдущий месяц.xlsx')
df.to_excel(writer)
writer.save()
print('Данные обработаны. Предыдущий месяц загружен в файл')

# Process the data for the month of the previous year corresponding to the current year.
df = pre_processed_df.copy()
df = df[['country', 'month_of_the_previous_year', 'new_code']]
df['Код ОКПД'] = ''
df['Параметр'] = 'Производство'
df['Месяц']    = month_of_the_previous_year_index
df['Год']      = previous_year
df['1, 2, 3']  = 3                       #  last year's month index
df = df.rename(columns={'country' : 'Субъект',
                        'month_of_the_previous_year' : 'Значение',
                        'new_code' : 'Код ОКПД-2'})
df = pd.DataFrame(df, columns=['Субъект',
                               'Код ОКПД',
                               'Параметр',
                               'Значение',
                               'Месяц',
                               'Год',
                               '1, 2, 3',
                               'Код ОКПД-2'])
df = df.dropna(subset=['Субъект'])
writer = pd.ExcelWriter('D:/Work/python/Преобразования через pandas/\
по работе/Обработанные файлы/Необходимые коды/Месяц предыдущего года.xlsx')
df.to_excel(writer)
writer.save()
print('Данные обработаны. Месяц прошлого года загружен в файл')

Текущий месяц          7 2021
Предыдущий месяц       6 2021
Месяц предыдущего года 7 2020
Данные обработаны. Текущий месяц загружен в файл
Данные обработаны. Предыдущий месяц загружен в файл
Данные обработаны. месяц предыдущего года загружен в файл
